# Code Along - Demonstration of GeoPandas

GeoPandas is a powerful Python Package for manipulating and plotting geographical data. In this notebook we will cover:

1. GeoPandas installation
2. Creating a GeoPandas DataFrame from a standard geographic file format
3. Setting the CRS (Coordinate Reference System)
4. Creating a map from the GeoPandas DataFrame.
5. Merging geographical areas.
6. Selecting small areas within a larger area.
7. Handling point data.
8. Converting geography between BNG (British National Grid Eastings and Northings) and Latitude/Longitude
9. Creating customisable maps with MatPlotLib

For geographic areas we most commonly use shape files. They frequently come contained within a Zip, along with other metadata files. The repository for this notebook (https://github.com/hsma5/5d_geopandas) contains all the necessary files. [UK government pages](https://geoportal.statistics.gov.uk/) to download these files tend to move around! At the time of checking (December 2022) the full England shape file download may be found at [here](https://geoportal.statistics.gov.uk/datasets/357ee15b1080431491bf965394090c72_0/explore?location=52.712816%2C-2.489527%2C6.98).

**Bonus data** You can also aceess estimated travel times for all LSOAs in England to 185 acute hospitals. Travel times are based on clear road conditions and can be found [here](https://gitlab.com/michaelallen1966/1811_lsoa_to_acute_hospital_travel).

## Installation

It is recommended that you install the environment as instructed in the README for this GitHub repo. You may also install packages manually in a different environment – proceed at your own risk (of compatibility issues)!

Alternatively save the following as a file titled `environment.yml`:

```
name: geopandas
channels:
  - defaults
  - conda-forge
dependencies:
  - geopandas
  - jupyterlab
  - mapclassify
  - pip
  - python=3.8
  - pip:
    - contextily
```

Then, from the folder where that yml file is, type the following into a terminal/command line: 

`conda env create -f environment.yml`

And then activate the environment with:

`conda activate geopandas`

And then launch Jupyter Lab with:

`jupyter-lab`

## Import modules

In [ ]:
import contextily as ctx
import geopandas
import matplotlib.pyplot as plt
import pandas as pd

import shapely
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 

## Load LSOA shape file

The following will load the shape file of the South West of England LSOA (Lower Super Output Areas) into a GeoPandas DataFrame. Note that we can set the CRS (Coordinate Reference System) when loading a GeoPandas DataFrame. EPSG:27700 is the CRS to use when geography is in BNG (British National Grid Eastings and Northings).

*Reminder - the three most common CRSs are:**
* **EPSG:27700** OSGB 1936 / British National Grid -- United Kingdom Ordnance Survey. Co-ordinates are in Eastings (X) and Northings (Y).
* **EPSG:4326** WGS 84 -- WGS84 - World Geodetic System 1984. Co-ordinates are in Longitude (X) and Latitude (Y).
* **EPSG:3857** -- Projected coordinate system used for rendering maps in Google Maps, OpenStreetMap, etc (i.e., projection for displaying lat/long as a flat map)

This file is a manually clipped section of the South West of England. In this notebook we will further restrict it to Devon and Cornwall.

[GeoPandas read_file documentation](https://geopandas.org/en/stable/docs/reference/api/geopandas.read_file.html)


In [ ]:
# Import England LSAO Shape file (use 'zip://'' prefix for zip files)
filename = "zip://../data/south_west_lsoa.zip"
lsoa_gdf = geopandas.read_file(filename, crs='EPSG:27700')

A GeoPandas DataFrame has a `geometry` column which contains the geographical details used for plotting and other geographic methods.

In [ ]:
lsoa_gdf.head(3)

We can create a simple plot using the `plot()` method for the Geopandas DataFrame.

In [ ]:
# The semi-colon supresses any additional info on screen
# i.e., <AxesSubplot:>
# NB the 8x8 is a max size, else would be distorted
lsoa_gdf.plot(figsize=(8,8));

<h3> <span style="color:red">Try the above steps in your PSGs - <em>15 mins.</em> </span></h3>

**Question:** Who can spot Lundy Island?

## Using Spatial Relationship methods to select data LSOAs in Devon or Cornwall

Here we will find LSOAs that are within the counties of Devon and Cornwall. Remember a LSOA is an area containing approx. [1500 people/ 650 households](https://ocsi.uk/2019/03/18/lsoas-leps-and-lookups-a-beginners-guide-to-statistical-geographies/), and so will vary in size (area). **Note: There is nothing in the geodataframe (gdf) above that explicitly states 'Devon' or 'Cornwall'.**

We will start by loading a file of geography for counties and unitary authorities.

In [ ]:
# Import County boundary shape file
# Originally from Government
filename = "zip://../data/Counties_and_Unitary_Authorities__December_2017" + \
    "__Boundaries_UK-shp.zip"

# Note - now performing same action as above - just in 2 distinct steps 
counties_gdf = geopandas.read_file(filename)

# CRS should be in the metadata of data file - but we can just try
counties_gdf = counties_gdf.set_crs(epsg=27700)

In [ ]:
# two lines of strings contact into one.
filename

In [ ]:
# 'ctyua' refers to County and Unitary Authority
counties_gdf.head()

We can see the boundries with the `plot()` method.

In [ ]:
# Note that these counties are national
# We can see boundaries around each county or unitary authority.
counties_gdf.plot(figsize=(8,8));

**Note: The County of Devon does not include Torbay or Plymouth Unitary Authorities: these will need to be included manually. [View listing here](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1026384/List_of_councils_in_England_2021.pdf).**

<h3> <span style="color:red">Try the above steps in PSGs and watch this video together -  <em>20 mins.</em> </span></h3>

### Lambda functions

Take a break and watch a 6 minute video on Lambda functions on YouTube https://youtu.be/25ovCm9jKfA

**Highly recommend Subscribing to Socratica (YouTube)**

In [ ]:
# Example of Lambda Function

import pandas as pd  
  
# assign data of lists.  
data = {'Name': ['Tom', 'Joseph', 'Krish', 'John'],
        'Age': [22, 21, 19, 18]}  
  
# Create DataFrame  
df = pd.DataFrame(data)  
  
# Print the output.
print("Original DF")
print(df)  

print('\n\n\n-----\n\n\n')
# Use lambda fn to populate new 'Age' col
df['over_21'] = df['Age'].apply(lambda x: 'yes' if x > 21 else 'no')

# Print the output.
print("Original DF")
print(df)  

### Finding the Devon and Cornwall data

The first thing we need to do is to identify the rows in the DataFrame that are in Cornwall and Devon. We can't just use county names as we must also include the unitary authorities 'Plymouth' and 'Torbay'.

The best way to apply a 'by row' function in DataFrames is to use the `map()` method. This applies a given function to all rows in a DataFrame. To check whether a value is in a list we need to apply a short function. As the function is simple we can use a *lambda* function as shown below.

Quick video on Map, Reduce, Filter - https://www.youtube.com/watch?v=hUes6y2b--0

<img src="../images/map_reduce_filter.png" alt="drawing" width="450"/>

In [ ]:
# Identify rows that are Devon or Cornwall (use Pandas map lambda function)
# Similar to going through each row of data in a 'for loop'
devon_cornwall = \
    counties_gdf['ctyua17nm'].map(
        lambda x: x in ['Devon', 'Cornwall', 'Plymouth', 'Torbay'])

In [ ]:
# Creates a pd.Series
devon_cornwall

In [ ]:
print(len(devon_cornwall))

In [ ]:
# Check number of 'True' within the Series
devon_cornwall.sum()

In [ ]:
# Add new column to DataFrame. This will be a booelan (True/False) column
# Could have done in single step
counties_gdf['devon_cornwall'] = devon_cornwall

In [ ]:
counties_gdf.head()

In [ ]:
mask = counties_gdf['devon_cornwall'] == True

In [ ]:
# Need to add .loc[] to look at rows
dc = counties_gdf.loc[mask]

In [ ]:
dc

In [ ]:
dc.plot(figsize=(8,8));

## Merging areas together

Now we are going to merge LSOAs by whether they are in Devon and Cornwall or not (we can merge on any column in order to combine areas). This will create a new DataFrame. To merge areas we use the `dissolve()` method.

https://geopandas.org/aggregation_with_dissolve.html

In [ ]:
# Create two groupings i.e., True and False re Devon and Cornwall
# 'devon_cornwall' is now the index column
# 'geometry' is now multipolygon (not just polygon)
merged_gdf = counties_gdf.dissolve(by='devon_cornwall')
merged_gdf

Now we select just the geometry Devon and Cornwall row, This has an index value of 'True'.

<a name="define_dc_geometry"> Defining `dc_geometry` </a>

In [ ]:
dc_geometry = merged_gdf.loc[True].geometry

Jupyter will display the geometry of a single geometry zone:
**This will take a moment......**

In [ ]:
# Remember that this is a geodataframe (not a regular dataframe)
# Jupyter knows that 'geometry' refers to geographic area - with CRS already applied.
dc_geometry

<h3> <span style="color:red">Try the above steps in PSGs -  <em>25 mins.</em> </span></h3>

### Spatial relationships

GeoPandas can perform a range of spacial relational tests such as those shown in the figure and table below (from https://en.wikipedia.org/wiki/Spatial_relation).

![](../images/spatial_fig.png)

![](../images/spatial_table.png)



To get all LSOAs in Devon or Cornwall we need to apply a test for `overlaps` or `within` (as `within` alone will miss those LSOAs that are share a boundary with the Devon and Cornwall region. In Pandas we use `|` for `or`.

In [ ]:
# Check for LSOA overlapping with, or completely within, Devon & Cornwall
# NOTE this code can take a moment to run....
mask = (lsoa_gdf.overlaps(dc_geometry)) | (lsoa_gdf.within(dc_geometry))

In [ ]:
mask

In [ ]:
# Create new Geopands DataFrame by applying mask
devon_cornwall_lsoa = lsoa_gdf.loc[mask]

In [ ]:
devon_cornwall_lsoa.head()

In [ ]:
# Show map of new DataFrame
devon_cornwall_lsoa.plot();

## Loading point data (hospitals) and selecting those in Devon and Cornwall.

We will load up a CSV file with hospital (acute stroke unit) data and create a geometry column from existing location data. This time our location data uses  Longitude and Latitude. For this we will use a different CRS, namely EPSG:4326 (also known as WGS84). This is a global reference system for Long/Lat. We will demonstrate an alternative way of setting CRS, setting it *after* we have created the GeoPandas DataFrame. 

In [ ]:
# Load hospital data in GeoPandas DataFrame
hospitals_gdf = geopandas.read_file('../data/hosp_107.csv')

In [ ]:
# See what's loaded...
# Take a look at the lat, long and geometry columns
hospitals_gdf

In [ ]:
# Create a populated 'geometry' field manually using the '.points_from_xy' method
# hospitals_gdf.long == hospitals_gdf['long']
hospitals_gdf.geometry = geopandas.points_from_xy(
        hospitals_gdf.long, hospitals_gdf.lat)

# Set CRS for geometry, using epsg4326 for lat/long
hospitals_gdf = hospitals_gdf.set_crs(epsg=4326)
hospitals_gdf.head()
# Note: geometry field is now a point (and not polygon/ multipolygon)

*Remember how we were initially working with BNG - then this hospital data was in lat/long?*

We now convert CRS from Lat/long to the BNG (EPSG 27700) we have been using in the LSOA DataFrame.

In [ ]:
hospitals_gdf = hospitals_gdf.to_crs(epsg=27700)

Notice how the geometry column has changed:

In [ ]:
# Take a look at the geometry column. What's different?
hospitals_gdf.head()

In [ ]:
# list of acute stroke units (eastings and northings)
hospitals_gdf.plot();

Identify hospitals `within` the Devon and Cornwall geometry zone we created earlier.

[Remind ourselves how we created `dc_geometry` here](#define_dc_geometry)

In [ ]:
mask = hospitals_gdf.within(dc_geometry)
dc_hospitals = hospitals_gdf.loc[mask]
dc_hospitals

<h3> <span style="color:red">Try the above steps -  <em>20 mins.</em> </span></h3>

## Bring in travel times to closest hospital

We will load data on travel times from each LSOA to their closest acute stroke unit. The DataFrame `merge` method is used to merge this into our GeoPandas DataFrame.

An application called [Routino](#https://www.routino.org/) may be used to get travel times - however it runs in Linux.

In [ ]:
# Import table of travel times from LSOA to closest stroke unit
travel_time = pd.read_csv('../data/lsoa_107_ivt.csv')

In [ ]:
travel_time.head()

In [ ]:
# Merge data
# Note we're interested in the `time_to_thrombolysis_unit` column
devon_cornwall_lsoa = devon_cornwall_lsoa.merge(
    travel_time[['area', 'time_to_thrombolysis_unit']], left_on='lsoa11nm', right_on='area', how='left')

In [ ]:
devon_cornwall_lsoa.head()

## Mapping data with MatPlotLib


Using MatPlotLib gives us more power than the GeoPandas DataFrame `plot` method.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10)) # Make max dimensions 10x10 inch
devon_cornwall_lsoa.plot(ax=ax, # Set which axes to use for plot (only one here)
                 column='time_to_thrombolysis_unit', # Column to apply colour
                 antialiased=False, # Avoids artifact boundry lines
                 edgecolor='face', # Make LSOA boundry same colour as area
                 vmin=0, # Manual scale min (remove to make automatic)
                 vmax=70, # Manual scale max (remove to make automatic)
                 cmap='inferno_r', # Colour map to use
                 # Adjust size of colourmap key, and add label
                 legend_kwds={'shrink':0.5, 'label':'Travel time (mins)'},
                 # Set to display legend
                 legend=True)
ax.set_axis_off() # Turn of axis linea dn numbers
plt.savefig('map_ca1.jpg', dpi=300) # Save figure
plt.show()

Repeat, but display hospitals as an extra plot using `ax` as axis. We will add hospital name (just postcode here), and use contexity to add a base map.

In [ ]:
# When using base maps convert data to epsg:3857
devon_cornwall_lsoa = devon_cornwall_lsoa.to_crs(epsg=3857)
dc_hospitals = dc_hospitals.to_crs(epsg=3857)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10)) # Make max dimensions 10x10 inch
# Plot travel times for each LSOA
devon_cornwall_lsoa.plot(ax=ax, # Set which axes to use for plot (only one here)
        column='time_to_thrombolysis_unit', # Column to apply colour
        # antialiasing loses transparency values
        # antialiased=False, # Avoids artifact boundry lines
        edgecolor='face', # Make LSOA boundry same colour as area
        linewidth=0.0,# Use linewidth=0 to hide boarder lines
        vmin=0, # Manual scale min (remove to make automatic)
        vmax=70, # Manual scale max (remove to make automatic)
        cmap='inferno_r', # Coloour map to use
        # Adjust size of colourmap key, and add label
        legend_kwds={'shrink':0.4, 'label':'Travel time (mins)'},
        # Set to display legend
        legend=True,
        # Set transparancy (to help reveal basemap)
        alpha = 0.70)

# Plot location of hospitals
dc_hospitals.plot(ax=ax, edgecolor='k', facecolor='w', markersize=200, 
                  marker='*')
# Add labels
for x, y, label in zip(
    dc_hospitals.geometry.x, dc_hospitals.geometry.y, dc_hospitals.hospital):
        ax.annotate(label, xy=(x, y), xytext=(8, 8), textcoords="offset points",
                    backgroundcolor="w", fontsize=8)
        
# Add base map (note that we specifiy thr same CRS as we are using)
# Use manual zoom to adjust level of detail of base map
ctx.add_basemap(ax, 
                source=ctx.providers.OpenStreetMap.Mapnik,zoom=10)
    
ax.set_axis_off() # Turn of axis line numbers
ax.set_title('Travel time (minutes) to closest acute stroke unit')
# Adjust for printing
ax.margins(0)
ax.apply_aspect()
plt.subplots_adjust(left=0.01, right=1.0, bottom=0.0, top=1.0)
# Save figure
plt.savefig('map_ca2.jpg', dpi=300)
plt.show()